## Custom gates

This plugin provides a set of custom gates that you can use in order to express your quantum circuits in MonarQ's native gate set. 

In [4]:
# this section is for initializing the document's code


# import custom gates
import pennylane_calculquebec.processing.custom_gates as custom

# import pennylane-specific functions
import pennylane as qml

import numpy as np
from pennylane.operation import Operation

dev = qml.device("default.qubit")


# this function will be used to print operation matrices
def print_matrix(op: Operation):
    print(op.name + " : ")
    print(f"{op.matrix()}\n")

### qml gates and custom gates

You might have noticed that we import ```pennylane as qml``` and ```custom_gates as custom```. 

That is because not all the native gates in MonarQ exist in pennylane. We thus have to add custom pennylane operations in order to cover all of MonarQ's native gate set. 

All the gates implemented by Calcul Québec reside in the ```custom_gates``` module. 

Here are the operations from MonarQ's native gate set : 

### one qubit gates

In [5]:
# a pi rad rotation around the X axis (has the effect of an amplitude flip)
x = qml.PauliX(0)
print_matrix(x)

PauliX : 
[[0 1]
 [1 0]]



In [6]:
# a pi rad rotation around the Z axis (has the effect of a phase flip)
z = qml.PauliZ(0)
print_matrix(z)

PauliZ : 
[[ 1  0]
 [ 0 -1]]



In [7]:
# a pi rad rotation around the Y axis (has the effect of combining a X and a Z)
y = qml.PauliY(0)
print_matrix(y)

PauliY : 
[[ 0.+0.j -0.-1.j]
 [ 0.+1.j  0.+0.j]]



In [8]:
# a pi/2 rad rotation around the Z axis
z90 = custom.Z90(0)
print_matrix(z90)

Z90 : 
[[0.70710678-0.70710678j 0.        +0.j        ]
 [0.        +0.j         0.70710678+0.70710678j]]



In [9]:
# a -pi/2 rad rotation around the Z axis
zm90 = custom.ZM90(0)
print_matrix(zm90)

ZM90 : 
[[0.70710678+0.70710678j 0.        +0.j        ]
 [0.        +0.j         0.70710678-0.70710678j]]



In [10]:
# a pi/2 rad rotation around the X axis
x90 = custom.X90(0)
print_matrix(z90)

Z90 : 
[[0.70710678-0.70710678j 0.        +0.j        ]
 [0.        +0.j         0.70710678+0.70710678j]]



In [11]:
# a -pi/2 rad rotation around the X axis
xm90 = custom.XM90(0)
print_matrix(zm90)

ZM90 : 
[[0.70710678+0.70710678j 0.        +0.j        ]
 [0.        +0.j         0.70710678-0.70710678j]]



In [12]:
# a pi/2 rad rotation around the Y axis
y90 = custom.Y90(0)
print_matrix(y90)

Y90 : 
[[ 0.70710678+0.j -0.70710678-0.j]
 [ 0.70710678+0.j  0.70710678+0.j]]



In [13]:
# a -pi/2 rad rotation around the Y axis
ym90 = custom.YM90(0)
print_matrix(ym90)

YM90 : 
[[ 0.70710678+0.j  0.70710678-0.j]
 [-0.70710678+0.j  0.70710678+0.j]]



In [14]:
# a pi/4 rad rotation around the Z axis
t = qml.T(0)
print_matrix(t)

T : 
[[1.        +0.j         0.        +0.j        ]
 [0.        +0.j         0.70710678+0.70710678j]]



In [15]:
# a -pi/4 rad rotation around the Z axis
tdag = custom.TDagger(0)
print_matrix(tdag)

TDagger : 
[[1.        -0.j         0.        +0.j        ]
 [0.        +0.j         0.70710678-0.70710678j]]



In [16]:
# an arbitrary rotation around the Z axis
p = qml.PhaseShift(2 * np.pi / 3, 0)
print_matrix(p)

PhaseShift : 
[[ 1. +0.j         0. +0.j       ]
 [ 0. +0.j        -0.5+0.8660254j]]



### 2 qubits gates

In [17]:
# if control (leftmost) qubit is set to 1, then target (rightmost) qubit will be applied a Z operation.
cz = qml.CZ([0, 1])
print_matrix(cz)

CZ : 
[[ 1  0  0  0]
 [ 0  1  0  0]
 [ 0  0  1  0]
 [ 0  0  0 -1]]



Here is an example of how you could use those operations in a circuit

In [ ]:
from pennylane_calculquebec.processing.config import EmptyConfig
from pennylane_calculquebec.API.client import CalculQuebecClient

# change the information in the client for your credentials
my_client = CalculQuebecClient(
    host="your host",
    user="your user",
    access_token="your access token",
    project_id="your project id",
)

# say you want to transpile your circuit by yourself
config = EmptyConfig()

# create a device that uses the empty configuration
dev = qml.device("monarq.default", client=my_client, processing_config=config)

dev.circuit_name = "your circuit"
dev.project = "your project"


dev.circuit_name = "your circuit"
dev.project = "your project"


# This would be equivalent to a bell state (Hadamard(0), CNOT([0, 1]))
# If we observe MonarQ's connectivity, we find that qubits 0 and 4 are adjacent
# If we search online a bit, we find that Hadamard(i) is equivalent to RZ(pi/2, i) - RX(pi/2, i) - RZ(pi/2, i)
# We also find that CNOT([i, j]) is equivalent to H(j) - CZ([i, j]) - H(j)
@qml.set_shots(1000)
@qml.qnode(dev)
def circuit():

    # native decomposition of a hadamard gate

    custom.Z90(0)
    custom.X90(0)
    custom.Z90(0)

    # native decomposition of a CNOT gate

    # Hadamard
    custom.Z90(1)
    custom.X90(1)
    custom.Z90(1)

    qml.CZ([0, 1])

    # Hadamard
    custom.Z90(1)
    custom.X90(1)
    custom.Z90(1)

    # apply readouts on qubits 0 and 1
    return qml.counts(wires=[0, 1])


# results should be the same as a bell state
results = circuit()
print(results)

{'0000': tensor(435, requires_grad=True), '0001': tensor(67, requires_grad=True), '0010': tensor(8, requires_grad=True), '0011': tensor(6, requires_grad=True), '0100': tensor(6, requires_grad=True), '0101': tensor(1, requires_grad=True), '1000': tensor(415, requires_grad=True), '1001': tensor(45, requires_grad=True), '1010': tensor(5, requires_grad=True), '1011': tensor(5, requires_grad=True), '1100': tensor(4, requires_grad=True), '1101': tensor(3, requires_grad=True)}


Even though there may be scenarios where it is beneficial to bypass the transpiling capabilites of the plugin, it is strongly recommended to stick to the default configuration. You don't have to pass any configuration to the device to use the default preset.

For more information  about pennylane operations, click [here](https://docs.pennylane.ai/en/stable/introduction/operations.html)